<div style="display: flex; background-color: RGB(255,114,0);" >

# PROJET - Cojoden
</div>

<div style="display: flex; background-color: blue;" >

# 0. Référentiel : Développer la gestion des données analytiques d’un projet d’application
</div>

<div style="display: flex; background-color: Green; padding: 7px;" >

### 0.1. Rapport
</div>

- La documentation des sources de données identifiées nécessaires au projet, en fonction des résultats de l’analyse exploratoire.
- Les visualisations synthétisant les caractéristiques du ou des jeux de données.
- Le script de nettoyage des données.
- Les éléments de modélisation des données - schémas, requêtes, etc.
- La procédure de mise en place du système de gestion de base de données analytique.
- Le script d’insertion des données dans le système de gestion de base de données.

<div style="display: flex; background-color: Green; padding: 7px;" >

### 0.2. Soutenance
</div>

- Développer la gestion des données analytiques d’un projet d’application
- La documentation des sources de données identifiées nécessaires au projet, en fonction des résultats de l’analyse exploratoire.
- Les visualisations synthétisant les caractéristiques du ou des jeux de données - screenshot ou via un applicatif.
- La stratégie de nettoyage des données.
- Les éléments de modélisation des données - schémas, requêtes, etc.
- La justification du ou des systèmes de gestion de bases de données analytique adaptés à la modélisation des données.
- La procédure de mise en place du système de gestion de base de données analytique.

<div style="display: flex; background-color: Green; padding: 7px;" >

### 0.3. Démonstration
</div>

La démonstration doit présenter les éléments suivant :
- Le nettoyage des données.
- L’insertion des données dans le système de gestion de base de données

<div style="display: flex; background-color: blue;" >

# 1.La mission
</div>

La Coopérative des Joyeux Denisoviens, la CoJoDen, une agence de voyages engagée vient de vous embaucher pour créer un prototype informatique.

La CoJoDen a pour objectif d'étoffer son offre et monter en gamme en proposant à ses clients des séjours culturels thématiques. Ces séjours seront définis dynamiquement en fonction des souhaits du client.

Par exemple notre nouvelle cliente italienne Mme Gherardini souhaite organiser un petit périple pour découvrir les oeuvres de Gaston Chaissac. Elle ne dispose que d'un weekend et voudrait limiter ses déplacements à deux musées. Si vous regardez dans https://www.pop.culture.gouv.fr/ et vous faites une recherche sur Gaston Chaissac (plusieurs auteurs avec des casses différentes), vous trouverez des oeuvres principalement sur Nantes et Colmar. Vous pourrez proposer un séjour à cette dame en lui indiquant le temps estimé pour aller d'un lieu à l'autre (à vous de trouver un moyen de récupérer le temps de trajet, ce denier point est optionnel dans un premier temps).

A partir d'un extrait la base de donnée Joconde accessible au grand public, vous modéliserez une base de données relationnelle MySQL qui regroupera les informations que vous jugerez nécessaires.

Vous peuplerez la base avec les oeuvres disponibles en nettoyant et en homogénéisant au maximum les infos.
Vous créerez un code python qui servira de démonstrateur sur quelques exemples simples.

### Livrables

- Faites un démonstrateur python de faisabilité avec quelques références (stockage et accès aux
données)
- Au niveau des livrables, voici ce qu'il faudra: Script python du démonstrateur, pdf de modèle de base (PAS DE NOTEBOOK)
Si on a le temps cette semaine vous ferez une présentation de 10 min de votre programme.
Si c'est trop court pour vous, on fera ça lors de notre prochaine semaine de formation

- NoSQL pour la sauvegarde des PDF et images

### Ressources

- https://www.data.gouv.fr/fr/datasets/collections-des-musees-de-france-base-joconde/


Nomenclature des champs :
Ce jeu de données comprend l’intégralité du catalogue collectif des musées de France. Les notices d’objets des musées de France sont de toute nature (archéologie, beaux-arts, ethnologie…).
Le détail de la nomenclature de la liste des champs de la base Joconde est téléchargeable ci-dessous, voir "Pièce jointe".

<div style="display: flex; background-color: blue;" >

# 2.Extraction des données
</div>

<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.1. Chargement des données
</div>

In [1]:
from os import getcwd
from os.path import join
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import csv
import re
import cojoden_extraction as cojex
from IPython.core.display import HTML
from cojoden_functions import convert_string_to_search_string, convert_df_string_to_search_string
from collections import Counter

In [2]:
# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = 0

# Récupère le répertoire du programme
file_path = getcwd() + "\\"
data_set_path = file_path + "dataset\\"

data_set_file_name = "cojoden_origine_clean.csv"

print(f"Current execution path : {file_path}")
print(f"Dataset path : {data_set_path}")

Current execution path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\
Dataset path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\


In [3]:
# Chargement et nettoyage général
df_origin = pd.read_csv(join(data_set_path,data_set_file_name), low_memory=False)
display(HTML(df_origin.head().to_html()))

,ref,titre,auteur,annee_debut,annee_fin,domaine,type_oeuvre,materiaux_technique,museo,nom_officiel_musee,latitude,longitude,geo_ville,geo_departement,geo_region,geo_ecole_pays,creation_lieux,largeur_cm,hauteur_cm,profondeur_cm,inscription_precisions,texte
0,00000000001,Le tableau représente au premier plan le Calavon vu vers l'aval,TORINY Gratien Désiré Huet De - (peintre),NaN,NaN,peinture,tableau,"toile, bois (peinture à l'huile)",M0945,musée municipal d'archéologie,43.875736,5.397687,Musées d'Apt,Vaucluse,Provence-Alpes-Côte d'Azur,NaN,Apt,25.0,37.5,0.0,"(français ), En bas à droite : H. de Toriny ; (français ), au revers : Manuscrit au crayon., 'Vue du Pont-Julien à Bonnieux'","Le tableau représente au premier plan le Calavon vu vers l'aval; au bord de la rivière sont arrêtés une charette à deux roues, deux animaux dételés et un personnage. Un peu plus loin, un cavalier (ou une cavalière ) fait boire son cheval dans la rivière. La rive droite de la rivière ressemble à une plage sableuse, plate et large ; elle est dominée par un petit escarpement rocheux. Sur la rive gauche on devine un berger menant son troupeau, alors que la toiture d'une maison se distingue derrière les monticules rocheux. Au second plan, le Pont-Julien lance ses trois arches par-dessus le lit de la rivière. Les deux piles centrales sont fondées sur des massifs qui se distinguent par leur couleur grise dans la masse brune du pont. La montée nord du pont prolonge la ligne du Luberon dont la masse, en arrière-plan, semble enveloppée d'une brume que le peintre a traitée en camaïeu violacé. Un ciel bleu occupant toute la moitié supérieure du tableau est traversé de quelques nuages à frange rosée."
1,00000000002,Panneau de mosaïque dans un cadre en bois,NaN,NaN,NaN,archéologie;céramique;arts décoratifs,mosaïque,"mortier de chaux, pierre indéterminée, colorant(s), pigment(s) artificiel(s) ou dérivé(s, bois (mosaïque)",M0945,musée municipal d'archéologie,43.875736,5.397687,Musées d'Apt,Vaucluse,Provence-Alpes-Côte d'Azur,NaN,NaN,82.0,0.0,0.0,NaN,"Panneau de mosaïque dans un cadre en bois. ; Fragment de mosaïque romaine (pavement de sol) à décor floral et géométrique polychrome, encadré d'entrelacs ; le tout dans un cadre en bois. Fiche ancienne d'André DUMOULIN : 'Fragment important de mosaïque romaine à petits cubes blancs, rouges et noirs. 2 zones : motif ornemental (4 étoiles à 4 pointes) et une fleur ; encadrement d'entrelacs.'"
2,00000000003,Sculpture en calcaire tendre local provenant du domaine du Clos,NaN,NaN,NaN,sculpture,Sculpture en calcaire tendre local provenant du domaine du Clos,calcaire (sculpture),M0945,musée municipal d'archéologie,43.875736,5.397687,Musées d'Apt,Vaucluse,Provence-Alpes-Côte d'Azur,NaN,NaN,0.0,118.0,0.0,Sur l'objet : Inscription manuscrite peinte,"Sculpture en calcaire tendre local provenant du domaine du Clos, occupé en partie par l'actuel jardin public d'APT. Cette sculpture se trouvait auparavant dans le hall de l'Hôtel de Ville. Statue grandeur nature (1,18 m, sans la tête), représentant une déesse-mère, une MATER, assise sur un coussin disposé sur un tabouret à quatre pieds simples, reliés par des traverses delphiniformes et recouvert d'un lourd tissu frangé (hauteur du siège : 45 cm). Les bras reposent sur les cuisses dans une attitude hiératique. Elle est vêtue d'une tunique à manches mi-longues plissées qui recouvre les pieds habillés de chaussures fermées. Un manteau ample, drapé, découvre l'épaule et le bras droit et s'enroule autour du bras gauche ; un pan retombe en plis entre les jambes. Des encoches indiquent la position d'attributs disparus anciennement : la main droite tenait une patère en signe d'offrande et la main gauche repliée (dont l'annulaire est orné d'une bague) semble tenir et relever un pan du manteau tandis que dans le creux du bras reposait vraisemblablement une corne d'abondance. Sur les genoux, dans le creux du manteau, étaient déposés soit des fruits, soit une corbeille remplie de fruits (encoche

In [4]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605157 entries, 0 to 605156
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ref                     605157 non-null  object 
 1   titre                   600532 non-null  object 
 2   auteur                  528887 non-null  object 
 3   annee_debut             263230 non-null  float64
 4   annee_fin               62097 non-null   float64
 5   domaine                 605156 non-null  object 
 6   type_oeuvre             598271 non-null  object 
 7   materiaux_technique     543702 non-null  object 
 8   museo                   605079 non-null  object 
 9   nom_officiel_musee      599077 non-null  object 
 10  latitude                605157 non-null  float64
 11  longitude               605157 non-null  float64
 12  geo_ville               605119 non-null  object 
 13  geo_departement         599077 non-null  object 
 14  geo_region          

<div style="display: flex; background-color: blue;" >

# SEPARATION des données
</div>

In [4]:
run_extraction = 0

<div style="display: flex; background-color: indigo;" >

#### METIERS
</div>

In [5]:
if run_extraction > 0:
    df_metiers= cojex.extract_metiers(df=df_origin, src_col_name="auteur", dest_col_name="metier", clean_function=cojex.clean_metier, separator="\)", dest_path=data_set_path, dest_file_name='metiers_uniques.csv', verbose=verbose)
else:
    df_metiers = pd.read_csv(join(data_set_path,'cojoden_metiers_uniques.csv'))
print(f"{df_metiers.shape} données chargées ------> {list(df_metiers.columns)}")
df_metiers

(230, 2) données chargées ------> ['search', 'metier']


,search,metier
0,ADAPTATEUR,adaptateur
1,ADMINISTRATEUR,administrateur
2,AFFICHISTE,affichiste
3,AMATEUR,amateur
4,AQUAFORTISTE,aquafortiste
...,...,...
225,TUILIER,tuilier
226,TYPOGRAPHE,typographe
227,VANNIER,vannier
228,VERNISSEUR,vernisseur


<div style="display: flex; background-color: indigo;" >

#### VILLES
</div>

In [6]:
if run_extraction > 0:
    df_villes = cojex.extract_villes(df=df_origin, dest_path=data_set_path, dest_file_name='villes_departement_region_pays.csv',verbose=verbose)
else:
    df_villes = pd.read_csv(join(data_set_path,'cojoden_villes_departement_region_pays.csv'))
    df_villes = df_villes.set_index("id")
print(f"{df_villes.shape} données chargées ------> {list(df_villes.columns)}")

(377, 6) données chargées ------> ['ville_search', 'ville', 'departement', 'region1', 'region2', 'pays']


In [8]:
df_villes[(df_villes['pays'].notna()) & (df_villes['pays']!='France')]

,ville_search,ville,departement,region1,region2,pays
id,,,,,,
50,BONN,Bonn,NaN,NaN,NaN,Allemagne
63,BRUXELLES,Bruxelles,NaN,NaN,NaN,Belgique
115,DUBLIN,Dublin,NaN,NaN,NaN,République d'Irlande
169,LE VATICAN,Le Vatican,NaN,NaN,NaN,Italie
181,LONDRES,Londres,NaN,NaN,NaN,Grande-Bretagne
188,LUXEMBOURG,Luxembourg,NaN,NaN,NaN,Luxembourg
285,ROME,Rome,NaN,NaN,NaN,Italie
296,SAINT DENIS DE LA REUNION,Saint-Denis de La Réunion,Réunion,NaN,NaN,La Réunion
343,TOKYO,Tokyo,NaN,NaN,NaN,Japon


In [9]:
df_villes[(df_villes['departement'].isna())]

,ville_search,ville,departement,region1,region2,pays
id,,,,,,
50,BONN,Bonn,NaN,NaN,NaN,Allemagne
61,BRIANCON,Briançon,NaN,NaN,NaN,France
63,BRUXELLES,Bruxelles,NaN,NaN,NaN,Belgique
73,CASTELLARE DI MERCURIO,Castellare-di-Mercurio,NaN,Corse,NaN,France
115,DUBLIN,Dublin,NaN,NaN,NaN,République d'Irlande
169,LE VATICAN,Le Vatican,NaN,NaN,NaN,Italie
181,LONDRES,Londres,NaN,NaN,NaN,Grande-Bretagne
188,LUXEMBOURG,Luxembourg,NaN,NaN,NaN,Luxembourg
285,ROME,Rome,NaN,NaN,NaN,Italie


<div style="display: flex; background-color: indigo;" >

#### MUSEES
</div>

In [7]:
if run_extraction > 0:
    df_musees = cojex.extract_musees(df=df_origin, df_villes=df_villes, dest_path=data_set_path, dest_file_name='musees.csv', verbose=verbose)
else:
    df_musees = pd.read_csv(join(data_set_path,'cojoden_musees.csv'))
print(f"{df_musees.shape} données chargées ------> {list(df_musees.columns)}")
df_musees

(482, 5) données chargées ------> ['museo', 'nom', 'ville', 'latitude', 'longitude']


,museo,nom,ville,latitude,longitude
0,M0811,musée Boucher de Perthes,1,50.107120,1.833193
1,M0465,musée de l'Ephèbe,2,43.298047,3.476702
2,M0087,musée municipal des beaux-arts,3,44.202895,0.615471
3,M0894,musée Granet,4,43.494315,5.410889
4,M0895,musée de l'atelier de Paul Cézanne,4,43.537816,5.446601
...,...,...,...,...,...
477,M1008,musée départemental de la Révolution française,374,45.087530,5.773234
478,M1137,musée Mainssieux,375,45.356047,5.579652
479,M0191,musée Zervos,364,47.464133,3.743610
480,M0022,musée Westercamp,376,49.038815,7.944344


<div style="display: flex; background-color: indigo;" >

#### AUTEURS
</div>

In [11]:
df_origin.columns

Index(['ref', 'titre', 'auteur', 'annee_debut', 'annee_fin', 'domaine',
       'type_oeuvre', 'materiaux_technique', 'museo', 'nom_officiel_musee',
       'latitude', 'longitude', 'geo_ville', 'geo_departement', 'geo_region',
       'geo_ecole_pays', 'creation_lieux', 'largeur_cm', 'hauteur_cm',
       'profondeur_cm', 'inscription_precisions', 'texte'],
      dtype='object')

In [8]:
if run_extraction > 0:
    df_artistes = cojex.extract_artistes(df=df_origin, dest_path=data_set_path, dest_file_name='artistes.csv', verbose=verbose)
else:
    df_artistes = pd.read_csv(join(data_set_path,'cojoden_artistes.csv'))
    df_artistes = df_artistes.set_index("id")
print(f"{df_artistes.shape} données chargées ------> {list(df_artistes.columns)}")
df_artistes

(46235, 3) données chargées ------> ['nom_search', 'nom_naissance', 'dit']


,nom_search,nom_naissance,dit
id,,,
2,A B,A B,NaN
3,A B ET CIE,A B & Cie,NaN
4,A B R,A B R,NaN
5,A BORD ET CIE,A. Bord et Cie,NaN
6,A C,A C,NaN
...,...,...,...
46232,ZYLBERBERG ADEK,ZYLBERBERG Adek,NaN
46233,ZYLBERBERG MARIE,ZYLBERBERG Marie,NaN
46234,ZYLVELT ADAM VAN,ZYLVELT Adam Van,NaN


<div style="display: flex; background-color: indigo;" >

#### OEUVRES
</div>

In [9]:
if run_extraction > 0:
    df_oeuvres = cojex.extract_oeuvres(df=df_origin, df_villes=df_villes, dest_path=data_set_path, dest_file_name='oeuvres.csv', verbose=0)
else:
    df_oeuvres = pd.read_csv(join(data_set_path,'cojoden_oeuvres.csv'), low_memory=False)
print(f"{df_oeuvres.shape} données chargées ------> {list(df_oeuvres.columns)}")
df_oeuvres

(605157, 13) données chargées ------> ['ref', 'titre', 'type', 'domaine', 'annee_debut', 'annee_fin', 'lieux_conservation', 'largeur_cm', 'hauteur_cm', 'profondeur_cm', 'creation_lieux', 'inscriptions', 'texte']


,ref,titre,type,domaine,annee_debut,annee_fin,lieux_conservation,largeur_cm,hauteur_cm,profondeur_cm,creation_lieux,inscriptions,texte
0,00000000001,Le tableau représente au premier plan le Calav...,tableau,peinture,NaN,NaN,M0945,25.0,37.5,0.0,16,"(français ), En bas à droite : H. de Toriny ; ...",Le tableau représente au premier plan le Calav...
1,00000000002,Panneau de mosaïque dans un cadre en bois,mosaïque,archéologie;céramique;arts décoratifs,NaN,NaN,M0945,82.0,0.0,0.0,0,NaN,Panneau de mosaïque dans un cadre en bois. ; F...
2,00000000003,Sculpture en calcaire tendre local provenant d...,Sculpture en calcaire tendre local provenant d...,sculpture,NaN,NaN,M0945,0.0,118.0,0.0,0,Sur l'objet : Inscription manuscrite peinte,Sculpture en calcaire tendre local provenant d...
3,00000000004,Sculpture monilithique en ronde-bosse,calcaire (sculpture) Sculpture monilithique en...,sculpture,NaN,NaN,M0945,32.0,50.0,0.0,0,Sur l'objet : Inscription manuscrite à l'encre...,"Sculpture monilithique en ronde-bosse, dans un..."
4,00000000005,statuette Marbre blanc,statuette,sculpture,NaN,NaN,M0945,14.0,28.0,0.0,0,NaN,Marbre blanc (marbre de Carrare ). Traces de p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
605152,M7051022442,plaque de verre Portrait de la jeune femme aux...,plaque de verre,ethnologie;photographie,NaN,NaN,M7051,9.0,0.0,12.0,0,NaN,Portrait de la jeune femme aux cheveux courts ...
605153,M7051022447,plaque de verre Notre personnage à cheveux cou...,plaque de verre,ethnologie;photographie,NaN,NaN,M7051,9.0,0.0,12.0,0,NaN,Notre personnage à cheveux courts porte cette ...
605154,M7051022448,plaque de verre Photo : jeune femme cheveux co...,plaque de verre,ethnologie;photographie,NaN,NaN,M7051,9.0,0.0,12.0,0,NaN,Photo : jeune femme cheveux courts a enlevé so...
605155,M7051030233,vue stéréoscopique Une jeune enfant en photo p...,vue stéréoscopique,ethnologie;photographie,NaN,NaN,M7051,4.5,0.0,11.0,0,NaN,Une jeune enfant en photo porte un petit béret...


<div style="display: flex; background-color: indigo;" >

#### CREER
</div>

In [10]:
if run_extraction > 0:
    df_creation_oeuvres = cojex.extract_creation_oeuvres(df=df_origin, df_artistes=df_artistes, dest_path=data_set_path, dest_file_name='creation_oeuvres.csv', verbose=verbose)
else:
    df_creation_oeuvres = pd.read_csv(join(data_set_path,'cojoden_creation_oeuvres.csv'), low_memory=False)
print(f"{df_creation_oeuvres.shape} données chargées ------> {list(df_creation_oeuvres.columns)}")
df_creation_oeuvres

(639238, 3) données chargées ------> ['oeuvre', 'artiste', 'role']


,oeuvre,artiste,role
0,00000000001,42739,PEINTRE
1,00000000013,26677,NaN
2,00000000014,35734,SCULPTEUR
3,00000041950,24732,VERRIER
4,00000041951,24732,CERAMISTE
...,...,...,...
639233,M0381001687,45687,GRAVEUR
639234,M0381001687,44625,DESSINATEUR
639235,M0381001687,34838,DESSINATEUR
639236,M0381001687,23087,DESSINATEUR


<div style="display: flex; background-color: indigo;" >

#### COMPOSER
</div>

In [ ]:
import cojoden_nettoyage as cojonet

In [6]:
if run_extraction > 0:
    df_materiaux, df_compose = cojex.extract_materiaux_and_composer(df=df_origin, dest_path=data_set_path, dest_file_name='materiaux_techniques.csv', verbose=verbose)
else:
    df_materiaux = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques.csv'))
    df_materiaux = df_materiaux.set_index("id")

    df_compose = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques_compose.csv'))

print(f"{df_materiaux.shape} données chargées ------> {list(df_materiaux.columns)}")
print(f"{df_compose.shape} données chargées ------> {list(df_compose.columns)}")
df_materiaux

[extract_materiaux_and_composer]	 INFO : (8462, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques.csv
[extract_materiaux_and_composer]	 INFO : (1247151, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques_compose.csv
(8462, 2) données chargées ------> ['mat_search', 'materiaux_technique']
(1247151, 2) données chargées ------> ['oeuvre', 'materiaux']


,mat_search,materiaux_technique
id,,
1,1 COULEUR,1 couleur
2,10,(10)
3,12,12
4,13,13
5,14,14
...,...,...
8458,ZINC ZINCOGRAPHIE,zinc (zincographie
8459,ZINCOGRAPHIE,zincographie (?
8460,ZINCOGRAPHIE COLORIE,zincographie (colorié)


In [22]:
df_compose.shape

(1247151, 2)

In [24]:
df_compose[df_compose["oeuvre"]=="M7051022442"]

,oeuvre,materiaux
407676,M7051022442,3354
989034,M7051022442,6621


In [26]:
df_compose[df_compose["oeuvre"]=="00000000001"]

,oeuvre,materiaux
107216,00000000001,778
1213950,00000000001,7954


In [27]:
df_compose[df_compose["oeuvre"]=="M0170002906"]

,oeuvre,materiaux
541232,M0170002906,4298
672884,M0170002906,5094
1247135,M0170002906,8456


<div style="display: flex; background-color: indigo;" >

#### DOMAINE et CONCERNER
</div>

In [ ]:
df_origin['domaine'].value_counts()

dessin                                                                                235107
peinture                                                                               66689
estampe                                                                                38680
sculpture                                                                              30266
photographie                                                                           13482
                                                                                       ...  
industrie lithique;industrie osseuse;agriculture - élevage;néolithique;archéologie         1
peinture;philatélie                                                                        1
horlogerie;ethnologie;métrologie - instruments de mesure;vie militaire                     1
sculpture;Afrique;ethnologie                                                               1
ethnologie;artisanat - industrie;vie militaire                        

In [ ]:
run_extraction = 1
if run_extraction > 0:
    df_domaine, df_concerner = cojex.extract_domaines_and_concerner(df=df_origin, dest_path=data_set_path, dest_file_name='domaines.csv', verbose=verbose)
else:
    df_concerner = pd.read_csv(join(data_set_path,'cojoden_domaines.csv'))
    df_concerner = df_concerner.set_index("id")

    df_concerner = pd.read_csv(join(data_set_path,'cojoden_domaines_concerner.csv'))

print(f"{df_domaine.shape} données chargées ------> {list(df_domaine.columns)}")
print(f"{df_concerner.shape} données chargées ------> {list(df_concerner.columns)}")
df_domaine

[extract_domaines_and_concerner]	 INFO : (142, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\domaines.csv
[extract_domaines_and_concerner]	 INFO : (868730, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\domaines_concerner.csv
(142, 2) données chargées ------> ['dom_search', 'domaine']
(868730, 2) données chargées ------> ['oeuvre', 'domaine']


,dom_search,domaine
id,,
1,AFRIQUE,afrique
2,AFRIQUE DU NORD,afrique du nord
3,AFRIQUE SUB SAHARIENNE,afrique sub-saharienne
4,AGE DU BRONZE,âge du bronze
5,AGE DU FER,âge du fer
...,...,...
138,VIE MILITAIRE,vie militaire
139,VIE POLITIQUE ADMINISTRATIVE,vie politique - administrative
140,VIE PROFESSIONNELLE,vie professionnelle


<div style="display: flex; background-color: blue;" >

# ANNEXE
</div>

<div style="display: flex; background-color: green;" >

## 2.1. Titre 2
</div>

<div style="display: flex; background-color: indigo;" >

#### Titre 3
</div>

<div style="display: flex; background-color: deeppink;" >

##### Titre4
</div>

<div style="display: flex; background-color: coral;" >

##### Titre 5
</div>